<a href="https://colab.research.google.com/github/Meenusj/Case_study/blob/main/cnn_with_allmini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sentence-transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 4.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
import pandas as pd

# Load the CSV file
csv_file_path = 'train.csv'
df = pd.read_csv(csv_file_path, sep=';')

# Convert 'account.type' to 0 and 1
df['account.type'] = df['account.type'].map({'human': 0, 'bot': 1})

# Filter for 1000 bots and 1000 humans
bots = df[df['account.type'] == 1].sample(1000, random_state=42)
humans = df[df['account.type'] == 0].sample(1000, random_state=42)

# Combine the samples
balanced_df = pd.concat([bots, humans])

# Shuffle the combined DataFrame
balanced_df = balanced_df.sample(frac=1, random_state=42).reset_index(drop=True)

# Count occurrences in the 'account.type' column
account_type_counts = balanced_df['account.type'].value_counts()
print("Account Type Counts:")
print(account_type_counts)

# Count occurrences in the 'class_type' column
class_type_counts = balanced_df['class_type'].value_counts()
print("\nClass Type Counts:")
print(class_type_counts)


Account Type Counts:
account.type
0    1000
1    1000
Name: count, dtype: int64

Class Type Counts:
class_type
human     1000
others     385
rnn        325
gpt2       290
Name: count, dtype: int64


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report
from sentence_transformers import SentenceTransformer
import matplotlib.pyplot as plt
import nltk

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

# Load MiniLM model
model_name = 'all-MiniLM-L6-v2'
model = SentenceTransformer(model_name)

# Step 1: Read and Prepare Data

X = balanced_df['text'].values
y = balanced_df['account.type'].values

# Encode labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Splitting data into train, validation, test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Step 2: Get MiniLM Embeddings
def get_minilm_embeddings(model, texts):
    embeddings = model.encode(texts, convert_to_tensor=True)
    return embeddings.cpu().numpy()  # Convert to NumPy array and move to CPU

X_train_embeddings = get_minilm_embeddings(model, X_train)
X_val_embeddings = get_minilm_embeddings(model, X_val)
X_test_embeddings = get_minilm_embeddings(model, X_test)

# Step 3: Define and compile your model with MiniLM Embeddings
input_shape = X_train_embeddings.shape[1:]

model_cnn = tf.keras.Sequential([
    tf.keras.layers.Input(shape=input_shape),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(len(label_encoder.classes_), activation='softmax')
])

model_cnn.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

# Step 4: Training
history = model_cnn.fit(X_train_embeddings, y_train,
                        epochs=10,
                        batch_size=64,
                        validation_data=(X_val_embeddings, y_val),
                        callbacks=[tf.keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True)])

# Step 5: Evaluation
loss, accuracy = model_cnn.evaluate(X_test_embeddings, y_test)
print(f'Test accuracy: {accuracy}')

# Confusion Matrix
y_pred = np.argmax(model_cnn.predict(X_test_embeddings), axis=1)
cm = confusion_matrix(y_test, y_pred)
print('Confusion Matrix:')
print(cm)


# Print Actual vs Predicted Outputs
actual_vs_predicted = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
print("Actual vs Predicted Outputs:")
print(actual_vs_predicted.head(30))  # Display the first 30 predictions

# Save the model
model_cnn.save('text_classification_model_with_minilm_cnn.h5')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Epoch 1/10
22/22 [==============================] - 2s 16ms/step - loss: 0.6897 - accuracy: 0.5200 - val_loss: 0.6792 - val_accuracy: 0.6867
Epoch 2/10
22/22 [==============================] - 0s 5ms/step - loss: 0.6735 - accuracy: 0.6143 - val_loss: 0.6516 - val_accuracy: 0.7167
Epoch 3/10
22/22 [==============================] - 0s 5ms/step - loss: 0.6354 - accuracy: 0.6921 - val_loss: 0.5952 - val_accuracy: 0.7533
Epoch 4/10
22/22 [==============================] - 0s 6ms/step - loss: 0.5788 - accuracy: 0.7271 - val_loss: 0.5267 - val_accuracy: 0.7933
Epoch 5/10
22/22 [==============================] - 0s 5ms/step - loss: 0.5319 - accuracy: 0.7600 - val_loss: 0.4774 - val_accuracy: 0.8033
Epoch 6/10
22/22 [==============================] - 0s 5ms/step - loss: 0.4911 - accuracy: 0.7743 - val_loss: 0.4512 - val_accuracy: 0.7900
Epoch 7/10
22/22 [==============================] - 0s 4ms/step - loss: 0.4572 - accuracy: 0.7936 - val_loss: 0.4370 - val_accuracy: 0.8000
Epoch 8/10
22/22 [=

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
import pandas as pd

# Load the CSV file
csv_file_path = 'train.csv'
df = pd.read_csv(csv_file_path, sep=';')

# Convert 'account.type' to 0 and 1
df['account.type'] = df['account.type'].map({'human': 0, 'bot': 1})

In [ ]:

# Count occurrences in the 'account.type' column
account_type_counts = df['account.type'].value_counts()
print("Account Type Counts:")
print(account_type_counts)

Account Type Counts:
account.type
0    10358
1    10354
Name: count, dtype: int64


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report
from sentence_transformers import SentenceTransformer
import matplotlib.pyplot as plt
import nltk

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

# Load MiniLM model
model_name = 'all-MiniLM-L6-v2'
model = SentenceTransformer(model_name)

# Step 1: Read and Prepare Data

X = df['text'].values
y = df['account.type'].values

# Encode labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Splitting data into train, validation, test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Step 2: Get MiniLM Embeddings
def get_minilm_embeddings(model, texts):
    embeddings = model.encode(texts, convert_to_tensor=True)
    return embeddings.cpu().numpy()  # Convert to NumPy array and move to CPU

X_train_embeddings = get_minilm_embeddings(model, X_train)
X_val_embeddings = get_minilm_embeddings(model, X_val)
X_test_embeddings = get_minilm_embeddings(model, X_test)

# Step 3: Define and compile your model with MiniLM Embeddings
input_shape = X_train_embeddings.shape[1:]

model_cnn = tf.keras.Sequential([
    tf.keras.layers.Input(shape=input_shape),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(len(label_encoder.classes_), activation='softmax')
])

model_cnn.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

# Step 4: Training
history = model_cnn.fit(X_train_embeddings, y_train,
                        epochs=10,
                        batch_size=64,
                        validation_data=(X_val_embeddings, y_val),
                        callbacks=[tf.keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True)])

# Step 5: Evaluation
loss, accuracy = model_cnn.evaluate(X_test_embeddings, y_test)
print(f'Test accuracy: {accuracy}')

# Confusion Matrix
y_pred = np.argmax(model_cnn.predict(X_test_embeddings), axis=1)
cm = confusion_matrix(y_test, y_pred)
print('Confusion Matrix:')
print(cm)


# Print Actual vs Predicted Outputs
actual_vs_predicted = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
print("Actual vs Predicted Outputs:")
print(actual_vs_predicted.head(30))  # Display the first 30 predictions

# Save the model
model_cnn.save('text_classification_model_with_minilm_cnn.h5')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Epoch 1/10
227/227 [==============================] - 2s 4ms/step - loss: 0.5614 - accuracy: 0.7117 - val_loss: 0.4455 - val_accuracy: 0.7895
Epoch 2/10
227/227 [==============================] - 1s 4ms/step - loss: 0.4451 - accuracy: 0.7880 - val_loss: 0.4102 - val_accuracy: 0.8021
Epoch 3/10
227/227 [==============================] - 1s 4ms/step - loss: 0.4081 - accuracy: 0.8086 - val_loss: 0.3940 - val_accuracy: 0.8133
Epoch 4/10
227/227 [==============================] - 1s 4ms/step - loss: 0.3879 - accuracy: 0.8194 - val_loss: 0.3847 - val_accuracy: 0.8146
Epoch 5/10
227/227 [==============================] - 1s 4ms/step - loss: 0.3697 - accuracy: 0.8314 - val_loss: 0.3824 - val_accuracy: 0.8114
Epoch 6/10
227/227 [==============================] - 1s 4ms/step - loss: 0.3534 - accuracy: 0.8367 - val_loss: 0.3774 - val_accuracy: 0.8130
Epoch 7/10
227/227 [==============================] - 1s 4ms/step - loss: 0.3375 - accuracy: 0.8478 - val_loss: 0.3738 - val_accuracy: 0.8136
Epoch 

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
import pandas as pd

# Load the CSV file
csv_file_path = 'test.csv'
df_test = pd.read_csv(csv_file_path, sep=';')
df_test.head()

# Convert 'account.type' to 0 and 1
df_test['account.type'] = df_test['account.type'].map({'human': 0, 'bot': 1})

# Count occurrences in the 'account.type' column
account_type_counts = df['account.type'].value_counts()
print("Account Type Counts:")
print(account_type_counts)

Account Type Counts:
account.type
bot      1280
human    1278
Name: count, dtype: int64


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report
from sentence_transformers import SentenceTransformer
import matplotlib.pyplot as plt
import nltk

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

# Load MiniLM model
model_name = 'all-MiniLM-L6-v2'
model = SentenceTransformer(model_name)

# Step 1: Read and Prepare Data

X = balanced_df['text'].values
y = balanced_df['account.type'].values

# Encode labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Splitting data into train and validation sets only
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)

# Step 2: Get MiniLM Embeddings
def get_minilm_embeddings(model, texts):
    embeddings = model.encode(texts, convert_to_tensor=True)
    return embeddings.cpu().numpy()  # Convert to NumPy array and move to CPU

X_train_embeddings = get_minilm_embeddings(model, X_train)
X_val_embeddings = get_minilm_embeddings(model, X_val)

# Step 3: Define and compile your model with MiniLM Embeddings
input_shape = X_train_embeddings.shape[1:]

model_cnn = tf.keras.Sequential([
    tf.keras.layers.Input(shape=input_shape),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(len(label_encoder.classes_), activation='softmax')
])

model_cnn.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

# Step 4: Training
history = model_cnn.fit(X_train_embeddings, y_train,
                        epochs=10,
                        batch_size=64,
                        validation_data=(X_val_embeddings, y_val),
                        callbacks=[tf.keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True)])

# Step 5: Load and Prepare Test Data
test_df = pd.read_csv('test.csv', sep=';')
test_df['account.type'] = test_df['account.type'].map({'human': 0, 'bot': 1})
X_test = test_df['text'].values
y_test = test_df['account.type'].values  # Use the values directly

X_test_embeddings = get_minilm_embeddings(model, X_test)

# Step 6: Evaluation
loss, accuracy = model_cnn.evaluate(X_test_embeddings, y_test)
print(f'Test accuracy: {accuracy}')

# Confusion Matrix
y_pred = np.argmax(model_cnn.predict(X_test_embeddings), axis=1)
cm = confusion_matrix(y_test, y_pred)
print('Confusion Matrix:')
print(cm)

# Classification Report
report = classification_report(y_test, y_pred, target_names=['human', 'bot'])
print('Classification Report:')
print(report)

# Print Actual vs Predicted Outputs
actual_vs_predicted = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
print("Actual vs Predicted Outputs:")
print(actual_vs_predicted.head(30))  # Display the first 30 predictions

# Save the model
model_cnn.save('text_classification_model_with_minilm_cnn.h5')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Epoch 1/10
22/22 [==============================] - 2s 17ms/step - loss: 0.6867 - accuracy: 0.5586 - val_loss: 0.6685 - val_accuracy: 0.6933
Epoch 2/10
22/22 [==============================] - 0s 8ms/step - loss: 0.6542 - accuracy: 0.6557 - val_loss: 0.6255 - val_accuracy: 0.7267
Epoch 3/10
22/22 [==============================] - 0s 8ms/step - loss: 0.6044 - accuracy: 0.7121 - val_loss: 0.5634 - val_accuracy: 0.7550
Epoch 4/10
22/22 [==============================] - 0s 8ms/step - loss: 0.5506 - accuracy: 0.7443 - val_loss: 0.5172 - val_accuracy: 0.7533
Epoch 5/10
22/22 [==============================] - 0s 9ms/step - loss: 0.4983 - accuracy: 0.7657 - val_loss: 0.4876 - val_accuracy: 0.7700
Epoch 6/10
22/22 [==============================] - 0s 9ms/step - loss: 0.4642 - accuracy: 0.7843 - val_loss: 0.4797 - val_accuracy: 0.7750
Epoch 7/10
22/22 [==============================] - 0s 6ms/step - loss: 0.4392 - accuracy: 0.7993 - val_loss: 0.4699 - val_accuracy: 0.7783
Epoch 8/10
22/22 [=

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
